#### Importing important data
In this code I am trying to import the data as efficient as possible.
The data I am particulary interested in can be found in the google sheet document "interesting data"
The categories that are defined so far are:
###### Demographics
1. Literacy
2. population density
3. slum population
4. Urban population share
###### Deaths
5. covid infections
6. covid deaths
7. dates
8. district id
###### Vaccination
9. site vaccination progress
10. first dose vaccination
11. dates
12. district id
###### Age
13. age per 5 year categories to make a map of where older people live

In [1]:
import pandas as pd
import numpy as np

In [2]:
#As always we have to specify to jupyter that we want to see all the outputs.
from IPython.core.interactiveshell import InteractiveShell
InteractiveShell.ast_node_interactivity = "all"
pd.options.display.max_rows = 20


In [3]:
# We need to be able to access the right folder. I put mine in the following folders. This can be adjusted easily
# Keep in mind that the 'r' in front of the string converts
# the string to a 'raw' string, bc of conflicts with \ and " ' characters

# Base directory
base_dir =       r"C:\Users\danie\OneDrive\Bureaublad\Coding\EPA introduction to datascience\Intro to datascience project"

# This section specifies the file name directories that I want to use
covid_folder =  r"\covid_data\covid\csv"
demog_folder =  r"\covid_data\demography\csv"
hosp_folder =  r"\covid_data\hospitals\csv"

# This section identifies the actual files
deaths_file =   r"\covid_infected_deaths_pc11.csv"
vacc_file =     r"\covid_vaccination.csv" #we cant use this because it contains new district numbering
demog_file =    r"\pc11_demographics_district.csv"
age_file =      r"\age_bins_district_t_pc11.csv"
hosp_file =     r"\dlhs4_hospitals_dist_pc11.csv"

deaths_dir = base_dir + covid_folder + deaths_file
vacc_dir =   base_dir + covid_folder + vacc_file #NA
demog_dir =  base_dir + demog_folder + demog_file
age_dir =    base_dir + demog_folder + age_file
hosp_dir =   base_dir + hosp_folder + hosp_file

dirlist = [deaths_dir, vacc_dir, demog_dir, age_dir, hosp_dir]


In [4]:
deaths_frame = pd.read_csv(deaths_dir)
vacc_frame   = pd.read_csv(vacc_dir)
demog_frame  = pd.read_csv(demog_dir)
age_frame    = pd.read_csv(age_dir)
hosp_frame   = pd.read_csv(hosp_dir)

framedict = {"Deaths" : deaths_frame, 
             "Vaccination" : vacc_frame,
             "Demographics" : demog_frame, 
             "Age" : age_frame,
             "Hospitals" : hosp_frame}

for key in framedict:
    print(key," has ",len(framedict[key])," entries")
    
#longer numbers represent the entries of daily entries. Entries around 640 contain district entries.

Deaths  has  263862  entries
Vaccination  has  210103  entries
Demographics  has  643  entries
Age  has  617  entries
Hospitals  has  552  entries


In [5]:
print("The keys of the dataframes are: \n")
for key, value in framedict.items():
    print(key, ":")
    print('length: ',len(framedict[key]))
    print(value.keys())

The keys of the dataframes are: 

Deaths :
length:  263862
Index(['pc11_state_id', 'pc11_district_id', 'date', 'total_cases',
       'total_deaths'],
      dtype='object')
Vaccination :
length:  210103
Index(['lgd_state_id', 'lgd_state_name', 'lgd_district_id',
       'lgd_district_name', 'date', 'total_individuals_registered',
       'total_sessions_conducted', 'total_sites', 'total_covaxin',
       'total_covishield', 'first_dose_admin', 'second_dose_admin', 'male_vac',
       'female_vac', 'trans_vac', 'state', 'district', 'bad_flg_covishield',
       'bad_flg_covaxin'],
      dtype='object')
Demographics :
length:  643
Index(['pc11_state_id', 'pc11_district_id', 'pc11_urb_share', 'pc11_slum_pop',
       'pc11_vd_area', 'pc11_td_area', 'pc11_tot_area', 'pc11_pop_dens',
       'pc11r_pca_tot_p', 'pc11u_pca_tot_p', 'pc11_pca_tot_p',
       'pc11r_pca_tot_m', 'pc11u_pca_tot_m', 'pc11_pca_tot_m',
       'pc11r_pca_tot_f', 'pc11u_pca_tot_f', 'pc11_pca_tot_f',
       'pc11r_pca_p_lit', 'p

In [6]:
## Cleaning Hospital data

# Some districts are missing in the hosp data. This makes sure they are in there as NaN
distr_col = demog_frame['pc11_district_id'] 
hosp_frame = pd.merge(hosp_frame, distr_col, on = 'pc11_district_id', how = 'right')

df = hosp_frame
# We only have to rename our district code to 'censuscode' 
df = df.rename(columns={"pc11_district_id": "censuscode"})

#andd drop wierd 999th district
df = df[df['censuscode'] != 999]
df = df[df['censuscode'] != 0]


In [7]:
# This is what the new frame looks like
hosp_frame = df
hosp_frame.head()
print("The size of the hosp_frame is ",len(hosp_frame))

,pc11_state_id,censuscode,dlhs4_dh_beds,dlhs4_dh_count,dlhs4_dh_staff,dlhs4_dh_icu_beds,dlhs4_chc_beds,dlhs4_chc_count,dlhs4_chc_staff,dlhs4_chc_beds_ven,dlhs4_phc_beds,dlhs4_phc_count,dlhs4_phc_staff,dlhs4_phc_pop,dlhs4_phc_beds_oxy,pc11_pca_tot_p,dlhs4_phc_mult,dlhs4_total_beds,dlhs4_total_staff,dlhs4_total_facilities
1,NaN,1,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,NaN,2,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,NaN,3,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,NaN,4,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
5,NaN,5,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


The size of the hosp_frame is  640


In [8]:
# Here we filter the dataframes on interesting variables that we want to use
interesting_dir = base_dir + r"\variablecodes.xlsm"
interesting_frame = pd.read_excel(interesting_dir)
interesting_frame

,label,dataset,code,folder,remarks
0,Literacy,Demographics,pc11_pca_p_lit,demography,NaN
1,population density,Demographics,pc11_pop_dens,demography,NaN
2,slum population,Demographics,pc11_slum_pop,demography,only for urban
3,Urban population share,Demographics,pc11_urb_share,demography,NaN
4,covid infections,Deaths,total_cases,covid,NaN
5,covid deaths,Deaths,total_deaths,covid,NaN
6,dates,Deaths,date,covid,NaN
7,district id,Deaths,lgd_district_id,covid,NaN
8,site vaccination progress,Vaccination,total_sites,covid,NaN
9,first dose vaccination,Vaccination,first_dose_admin,covid,NaN


In [9]:
#Framedict['Deaths'] is the frame of casualties and cases per day.
framedict['Deaths'].head()


,pc11_state_id,pc11_district_id,date,total_cases,total_deaths
0,1,1,30jan2020,0.0,0.0
1,1,1,02feb2020,0.0,0.0
2,1,1,03feb2020,0.0,0.0
3,1,1,02mar2020,0.0,0.0
4,1,1,03mar2020,0.0,0.0


In [10]:
df = framedict['Deaths']
# We only have to rename our district code to 'censuscode' 
df = df.rename(columns={"pc11_district_id": "censuscode"})

#andd drop wierd 999th district
df = df[df['censuscode'] != 999]
df = df[df['censuscode'] != 0]
deaths_frame = df


In [11]:
len(df)
df.head()

250710

,pc11_state_id,censuscode,date,total_cases,total_deaths
0,1,1,30jan2020,0.0,0.0
1,1,1,02feb2020,0.0,0.0
2,1,1,03feb2020,0.0,0.0
3,1,1,02mar2020,0.0,0.0
4,1,1,03mar2020,0.0,0.0


In [12]:
# We can use this dataframe to filter on specific dates.
# This code does that:
def stripframe(df, date):
    if type(date) != list:
        date = [date]
    retdict = {}
    for i in date:
        newframe = df[df['date'] == i] 
        # This dataframe has to be pivotted to conform to the district name on the left.
        df2 = newframe.pivot_table(index = newframe.index, columns = [])
    
        retdict.update({i : df2})
    return retdict

# this code is an example on how to extract dates from this dataframe.

dates = ['28mar2021', '04apr2021', '11apr2021']
#dates = '28mar2021'

datedict = stripframe(df, dates)    

#check if the data has been parsed correctly.
print('Lengths of the dataframes are: ', [(i, len(j)) for i,j in datedict.items()])
overview = [print('\n'+str(i)+'\n',j) for i,j in datedict.items()]


Lengths of the dataframes are:  [('28mar2021', 610), ('04apr2021', 610), ('11apr2021', 610)]

28mar2021
         censuscode  pc11_state_id  total_cases  total_deaths
394              1              1       5791.0          97.0
805              2              1       8145.0         120.0
1216             3              1       7828.0          86.0
1627             4              1       2146.0          44.0
2038             5              1       2542.0          25.0
...            ...            ...          ...           ...
261379         635             34      32325.0         550.0
261790         636             34       2303.0          10.0
262201         637             34       4331.0          75.0
263023         639             35          0.0           0.0
263434         640             35          0.0           0.0

[610 rows x 4 columns]

04apr2021
         censuscode  pc11_state_id  total_cases  total_deaths
401              1              1       5857.0          97.0
812  

In [13]:
framedict = {"Deaths" : deaths_frame, 
             "Vaccination" : vacc_frame,
             "Demographics" : demog_frame, 
             "Age" : age_frame,
             "Hospitals" : hosp_frame}